In [5]:
import json
import pandas as pd
import os


In [6]:
class IPLdata:
    def __init__(self, season, city, venue, date, match_type, match_number, teams, toss_winner, toss_decision, outcome_by_runs, outcome_by_wickets, winner, player_of_match, target, innings):
        self.season = season
        self.city = city
        self.venue = venue
        self.date = date
        self.match_type = match_type
        self.match_number = match_number
        self.teams = teams
        self.toss_winner = toss_winner
        self.toss_decision = toss_decision
        self.outcome_by_runs = outcome_by_runs
        self.outcome_by_wickets = outcome_by_wickets
        self.winner = winner
        self.player_of_match = player_of_match
        self.target = target
        self.innings = innings

    def to_dict(self):
        return{
            'season': self.season,
            'city': self.city,
            'venue': self.venue,
            'date': self.date,
            'match_type': self.match_type,
            'match_number': self.match_number,
            'teams': self.teams,
            'toss_winner': self.toss_winner,
            'toss_decision': self.toss_decision,
            'outcome_by_runs': self.outcome_by_runs,
            'outcome_by_wickets': self.outcome_by_wickets,
            'winner': self.winner,
            'player_of_match': ', '.join(self.player_of_match),
            'target': self.target
        }    

    def innings_dict(self):
        innings_data = []
        for inning in self.innings:
            team = inning['team']
            for over in inning['overs']:
                for delivery in over['deliveries']:
                    innings_data.append({
                        'season': self.season,
                        'match_number': self.match_number,
                        'team': team,
                        'over': over['over'],
                        'batter': delivery['batter'],
                        'bowler': delivery['bowler'],
                        'runs_batter': delivery['runs'].get('batter', 0),
                        'runs_extras': delivery['runs'].get('extras', 0),
                        'runs_total': delivery['runs'].get('total', 0),
                        'non_striker': delivery['non_striker'],

                    })
        return innings_data       



In [7]:
class IPLdataloader:
    def __init__(self,folder_path):
        self.folder_path = folder_path
        self.matches = []

    def load_data(self):
        if not os.path.exists(self.folder_path):
            print(f"Error: The folder {self.folder_path} does not exist.")
            return
        
        json_files = [f for f in os.listdir(self.folder_path) if f.endswith('.json')]

        if not json_files:
            print(f"Error: No JSON files found in the folder {self.folder_path}.")
            return
        
        for json_file in json_files:
            try:
                file_path = os.path.join(self.folder_path, json_file)
                with open(file_path, 'r') as file:
                    data = json.load(file)

                    season = data['info'].get('season', 'N/A')  
                    city = data['info'].get('city', 'N/A')
                    venue = data['info'].get('venue', 'N/A')
                    date = data['info']['dates'][0] if 'dates' in data['info'] else 'N/A'
                    match_type = data['info'].get('match_type', 'N/A')
                    match_number = data['info']['event'].get('match_number', 'N/A')
                    teams = data['info'].get('teams', [])
                    toss_winner = data['info']['toss'].get('winner', 'N/A') if 'toss' in data['info'] else 'N/A'
                    toss_decision = data['info']['toss'].get('decision', 'N/A') if 'toss' in data['info'] else 'N/A'
                    #outcome = f"Winner: {data['info'].get('outcome', {}).get('winner', 'N/A')}, By {data['info'].get('outcome', {}).get('by', {}).get('runs', 0)} runs"
                    outcome_by_runs = data['info']['outcome']['by'].get('runs', 'N/A') if 'by' in data['info']['outcome'] else 'N/A'
                    outcome_by_wickets = data['info']['outcome']['by'].get('wickets', 'N/A') if 'by' in data['info']['outcome'] else 'N/A'
                    winner = data['info']['outcome'].get('winner','N/A')
                    player_of_match = data['info'].get('player_of_match', [])
                    target= data['innings'][1].get('target', {}).get('runs', 'N/A')  
                    innings = data.get('innings', [])
                    
                    # Create an instance of CricketMatch
                    match = IPLdata(season, city, venue, date, match_type, match_number,teams, toss_winner, toss_decision, outcome_by_runs, outcome_by_wickets, winner, player_of_match, target, innings)
                    self.matches.append(match)

            except PermissionError:
                print(f"Error: Permission denied while trying to open the file {json_file}. Please check your file permissions.")
            except Exception as e:
                print(f"An unexpected error occurred with file {json_file}: {e}")

    def to_dataframe(self):
        # Convert list of CricketMatch objects to a pandas DataFrame for match data
        match_data = [match.to_dict() for match in self.matches]
        match_df = pd.DataFrame(match_data)

        # Convert innings data to a DataFrame
        innings_data = []
        for match in self.matches:
            innings_data.extend(match.innings_dict())
        innings_df = pd.DataFrame(innings_data)

        return match_df, innings_df    






In [8]:
folder_path = 'C:/Users/Sajiv/Documents/Cricket/ipl'  

print(f"Current working directory: {os.getcwd()}")

loader = IPLdataloader(folder_path)
loader.load_data()

if loader.matches:
    match_df, innings_df = loader.to_dataframe()

    # Displaying the DataFrames
    print("\nMatch DataFrame:")
    print(match_df)

    print("\nInnings DataFrame:")
    print(innings_df)
else:
    print("No data loaded. Please check the error messages above.")

Current working directory: c:\Users\Sajiv\Documents\Cricket\Notebook
An unexpected error occurred with file 1359519.json: list index out of range
An unexpected error occurred with file 501265.json: list index out of range
An unexpected error occurred with file 829763.json: list index out of range

Match DataFrame:
     season       city                                             venue  \
0      2017  Hyderabad         Rajiv Gandhi International Stadium, Uppal   
1      2017       Pune           Maharashtra Cricket Association Stadium   
2      2017     Rajkot            Saurashtra Cricket Association Stadium   
3      2017     Indore                            Holkar Cricket Stadium   
4      2017  Bengaluru                             M.Chinnaswamy Stadium   
...     ...        ...                                               ...   
1087   2016     Raipur  Shaheed Veer Narayan Singh International Stadium   
1088   2016  Bangalore                             M Chinnaswamy Stadium   


In [9]:
match_df.head(6)

,season,city,venue,date,match_type,match_number,teams,toss_winner,toss_decision,outcome_by_runs,outcome_by_wickets,winner,player_of_match,target
0,2017,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",2017-04-05,T20,1,"[Sunrisers Hyderabad, Royal Challengers Bangal...",Royal Challengers Bangalore,field,35,N/A,Sunrisers Hyderabad,Yuvraj Singh,208
1,2017,Pune,Maharashtra Cricket Association Stadium,2017-04-06,T20,2,"[Rising Pune Supergiant, Mumbai Indians]",Rising Pune Supergiant,field,N/A,7,Rising Pune Supergiant,SPD Smith,185
2,2017,Rajkot,Saurashtra Cricket Association Stadium,2017-04-07,T20,3,"[Gujarat Lions, Kolkata Knight Riders]",Kolkata Knight Riders,field,N/A,10,Kolkata Knight Riders,CA Lynn,184
3,2017,Indore,Holkar Cricket Stadium,2017-04-08,T20,4,"[Kings XI Punjab, Rising Pune Supergiant]",Kings XI Punjab,field,N/A,6,Kings XI Punjab,GJ Maxwell,164
4,2017,Bengaluru,M.Chinnaswamy Stadium,2017-04-08,T20,5,"[Royal Challengers Bangalore, Delhi Daredevils]",Royal Challengers Bangalore,bat,15,N/A,Royal Challengers Bangalore,KM Jadhav,158
5,2017,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",2017-04-09,T20,6,"[Sunrisers Hyderabad, Gujarat Lions]",Sunrisers Hyderabad,field,N/A,9,Sunrisers Hyderabad,Rashid Khan,136


In [10]:
match_df.to_csv("C:/Users/Sajiv/Documents/Cricket/IPL_set/IPL_match_info.csv")

In [11]:
innings_df.head(6)

,season,match_number,team,over,batter,bowler,runs_batter,runs_extras,runs_total,non_striker
0,2017,1,Sunrisers Hyderabad,0,DA Warner,TS Mills,0,0,0,S Dhawan
1,2017,1,Sunrisers Hyderabad,0,DA Warner,TS Mills,0,0,0,S Dhawan
2,2017,1,Sunrisers Hyderabad,0,DA Warner,TS Mills,4,0,4,S Dhawan
3,2017,1,Sunrisers Hyderabad,0,DA Warner,TS Mills,0,0,0,S Dhawan
4,2017,1,Sunrisers Hyderabad,0,DA Warner,TS Mills,0,2,2,S Dhawan
5,2017,1,Sunrisers Hyderabad,0,S Dhawan,TS Mills,0,0,0,DA Warner


In [12]:
innings_df.to_csv("C:/Users/Sajiv/Documents/Cricket/IPL_set/IPL_innings.csv")